In [1]:
import os
# Find the latest version of spark 3.2 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.2.3'
spark_version = 'spark-3.2.4'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Get:3 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease [3,622 B]
Hit:4 http://archive.ubuntu.com/ubuntu focal InRelease
Get:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease [18.1 kB]
Get:6 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Hit:7 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Hit:8 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Get:9 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Get:10 http://security.ubuntu.com/ubuntu focal-security/main amd64 Packages [2,681 kB]
Get:11 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease [24.3 kB]
Get:12 http://security.ubuntu.com/ubuntu focal-security/universe amd64 Packages [1,046 kB]
Hit:13 http://ppa.launchpad.net/ubuntugis/ppa/ubuntu focal InRe

In [2]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2023-05-13 21:04:22--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  6.16MB/s    in 0.2s    

2023-05-13 21:04:22 (6.16 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M17-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [4]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Digital_Video_Games_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+-------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|   product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+-------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   21269168| RSH1OZ87OYK92|B013PURRZW|     603406193|Madden NFL 16 - X...|Digital_Video_Games|          2|            2|          3|   N|                N|A slight improvem...|I keep buying mad...| 2015-08-31|
|         US|     133437|R1WFOQ3N9BO65I|B00F4CEHNK|     341969535| Xbox Live Gift Card|Digital_Video_Games| 

In [5]:
from pyspark.sql.functions import to_date
# Read in the Review dataset as a DataFrame
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Digital_Video_Games_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("amazon_reviews_us_Digital_Video_Games_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True, timestampFormat="mm/dd/yy")
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+-------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|   product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+-------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   21269168| RSH1OZ87OYK92|B013PURRZW|     603406193|Madden NFL 16 - X...|Digital_Video_Games|          2|            2|          3|   N|                N|A slight improvem...|I keep buying mad...| 2015-08-31|
|         US|     133437|R1WFOQ3N9BO65I|B00F4CEHNK|     341969535| Xbox Live Gift Card|Digital_Video_Games| 

In [6]:
# Recreate the vine_table. DataFrame
vine_df = df.select(['review_id', 'star_rating', 'helpful_votes', 'total_votes','vine','verified_purchase'])
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| RSH1OZ87OYK92|          2|            2|          3|   N|                N|
|R1WFOQ3N9BO65I|          5|            0|          0|   N|                Y|
| R3YOOS71KM5M9|          5|            0|          0|   N|                Y|
|R3R14UATT3OUFU|          5|            0|          0|   N|                Y|
| RV2W9SGDNQA2C|          5|            0|          0|   N|                Y|
|R3CFKLIZ0I2KOB|          5|            0|          0|   N|                Y|
|R1LRYU1V0T3O38|          4|            0|          0|   N|                Y|
| R44QKV6FE5CJ2|          5|            0|          0|   N|                Y|
|R2TX1KLPXXXNYS|          5|            0|          0|   N|                Y|
|R1JEEW4C6R89BA|          5|            0|          0|   N|     

1: Filter the data and create a new DataFrame or table to retrieve all the rows where the total_votes count is equal to or greater than 20 to pick reviews that are more likely to be helpful and to avoid having division by zero errors later on.

In [9]:
filtered1_df = vine_df.filter('total_votes>20')
filtered1_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| RV4UH0OWW97K4|          5|           19|         22|   N|                N|
| RG6GT6G9WNYT1|          5|           22|         22|   N|                N|
| RFBXE3Q3GYQYO|          5|           68|         70|   N|                Y|
|R235HEIYY15NZX|          1|            0|         26|   N|                Y|
|R3BZ4DHX2YHC4V|          1|            1|         32|   N|                Y|
| RZSC8Q4630BV1|          1|            1|         26|   N|                N|
|R34S64NPUZOS90|          5|           23|         24|   N|                N|
|R35O3GV3HZUX8B|          5|           58|         63|   N|                N|
|R2AZAMZCEUOKQT|          1|           29|         40|   N|                Y|
|R103RUVEH5YWQ4|          1|           69|         90|   N|     

2: Filter the new DataFrame or table created in Step 1 and create a new DataFrame or table to retrieve all the rows where the number of helpful_votes divided by total_votes is equal to or greater than 50%

In [10]:
filtered2_df = filtered1_df.filter('helpful_votes/total_votes>=0.5')
filtered2_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| RV4UH0OWW97K4|          5|           19|         22|   N|                N|
| RG6GT6G9WNYT1|          5|           22|         22|   N|                N|
| RFBXE3Q3GYQYO|          5|           68|         70|   N|                Y|
|R34S64NPUZOS90|          5|           23|         24|   N|                N|
|R35O3GV3HZUX8B|          5|           58|         63|   N|                N|
|R2AZAMZCEUOKQT|          1|           29|         40|   N|                Y|
|R103RUVEH5YWQ4|          1|           69|         90|   N|                Y|
|R2QNPQLXMCXOH3|          5|           35|         37|   N|                Y|
| R1JENRRBQ1YMY|          5|           35|         36|   N|                N|
|R2NS0HPQ2DD5XU|          1|           35|         45|   N|     

3: Filter the DataFrame or table created in Step 2, and create a new DataFrame or table that retrieves all the rows where a review was written as part of the Vine program (paid), vine == 'Y'.

In [11]:
filtered3_df = filtered2_df.filter('vine=="Y"')
filtered3_df.show()

+---------+-----------+-------------+-----------+----+-----------------+
|review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+---------+-----------+-------------+-----------+----+-----------------+
+---------+-----------+-------------+-----------+----+-----------------+



4: Repeat Step 3, but this time retrieve all the rows where the review was not part of the Vine program (unpaid), vine == 'N'.

In [13]:
filtered4_df = filtered2_df.filter('vine=="N"')
filtered4_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| RV4UH0OWW97K4|          5|           19|         22|   N|                N|
| RG6GT6G9WNYT1|          5|           22|         22|   N|                N|
| RFBXE3Q3GYQYO|          5|           68|         70|   N|                Y|
|R34S64NPUZOS90|          5|           23|         24|   N|                N|
|R35O3GV3HZUX8B|          5|           58|         63|   N|                N|
|R2AZAMZCEUOKQT|          1|           29|         40|   N|                Y|
|R103RUVEH5YWQ4|          1|           69|         90|   N|                Y|
|R2QNPQLXMCXOH3|          5|           35|         37|   N|                Y|
| R1JENRRBQ1YMY|          5|           35|         36|   N|                N|
|R2NS0HPQ2DD5XU|          1|           35|         45|   N|     

5: Determine the total number of reviews, the number of 5-star reviews, and the percentage of 5-star reviews for the two types of review (paid vs unpaid).

In [16]:
from pyspark.sql.functions import count

#Paid
p_total = filtered3_df.count()
p_fivestar = filtered3_df.filter(filtered3_df.star_rating==5).count()
#p_fivestarPercent = (p_fivestar/p_total)*100

#Unpaid
u_total = filtered4_df.count()
u_fivestar = filtered4_df.filter(filtered4_df.star_rating==5).count()
u_fivestarPercent = (u_fivestar/u_total)*100


In [29]:
#Paid
print("paid total count is", p_total)
print("paid five stars count is", p_fivestar)
print("paid five stars percentage is","[DIVIDE BY ZERO ERROR]")

paid total count is 0
paid five stars count is 0
paid five stars percentage is [DIVIDE BY ZERO ERROR]


In [27]:
#Unpaid
print("unpaid total count is", u_total)
print("unpaid five stars count is", u_fivestar)
print("unpaid five stars percentage is", u_fivestarPercent)

unpaid total count is 1563
unpaid five stars count is 582
unpaid five stars percentage is 37.23608445297505
